<a href="https://colab.research.google.com/github/HausReport/ClubRaiders/blob/master/ClubRaiders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![banner](https://raw.githubusercontent.com/HausReport/ClubRaiders/master/info/wordcloud2.png)

# Introduction

# Number Crunching

In [0]:
#@title
import sys
sys.executable
sys.path

%load_ext google.colab.data_table
!{sys.executable} -m pip install --upgrade --force-reinstall "git+https://github.com/HausReport/ClubRaiders.git#egg=craid"

from dateutil.relativedelta import relativedelta
import datetime
from datetime import timezone 

!jupyter nbextension enable --py widgetsnbextension
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

#import cufflinks as cf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px



In [0]:
#@title 
import craid.Club
import craid.LoadDataFromEDDB
from craid.eddb.FactionInstance import FactionInstance
import craid.Crunch
craid.LoadDataFromEDDB.load_data()
ret = craid.Crunch.getSystemsArray()
club_systems_arr: FactionInstance = ret[0]

df = craid.Crunch.getDataFrame(club_systems_arr)
#
# Dataframe of all club factions except Emp Grace
#
filter = df[ ~df[ 'factionName' ].str.contains("Emperor's Grace") ]



# Intercepted Message

In [0]:
#@title 


frame = df

dt = datetime.datetime.now() 
utc_time = dt.replace(tzinfo = timezone.utc) 

new_date = utc_time + relativedelta(years=1)
print("TO: CMDR [REDACTED]/OPS")
print("FM: CMDR E[REDACTED]/MAIA DATA")
print("DT: " + new_date.strftime("%d-%b-%Y %H:%M"))
print("RE: HAXDAT/CLUB STUFF TO SHOOT")
print("===================================")

print()
print("Hey.  Spent the last two nights up to my ears in data from the last breach.  Here are the ")
print("highlights.  I think they're pretty compelling, but I'm juiced to the gills to stay awake.")

print()

print( "Club factions are currently active in " + str(frame['systemName'].count()) +" systems, home to " + '{:,}'.format(int(frame['population'].sum(axis=0))) + " souls." )
idx = frame['x'].idxmin()

print('They span the galaxy from {0} in {1}, {2} light years west of Sol'.format(frame.iloc[idx,1], frame.iloc[idx,0], abs(int(frame.iloc[idx,2]))))
idx = frame['x'].idxmax()

print('to {0} in {1}, {2} light years to the east.'.format(frame.iloc[idx,1], frame.iloc[idx,0], abs(int(frame.iloc[idx,2]))))

print()

print("It's encouraging to note that club factions only control {0} of these systems, or about {1:.0%}.".format(frame['control'].sum(), frame['control'].sum()/frame['systemName'].count()))
print("Their influence in all systems they operate in ranges from {0:.0%} to {1:.0%}, averaging {2:.0%}, but".format(df['influence'].min()/100.0, df['influence'].max()/100.0, df['influence'].mean()/100.0))

uncontrolled = frame[ ~frame[ 'control' ]]

print("in systems they don't control, these numbers fall to {0:.0%}-{1:.0%} influence with an average of only {2:.0%}.".format(uncontrolled['influence'].min()/100.0, uncontrolled['influence'].max()/100.0, uncontrolled['influence'].mean()/100.0))

gaxx =uncontrolled['influence'].describe()

print("In these non-control systems, half of their factions have less than {0:.0%} influence and a full\nquarter are operating at less than {1:.0%}.".format(gaxx[5]/100,gaxx[4]/100))

print()
print("As you know, we forced a number of club retreats lately.  Here's today's hot list:")
print()

gg = filter.copy()
gg = gg[gg['influence']<6]
gg = gg[gg['influence']>0]
gg = gg[~gg['isHomeSystem']]
gg=  gg.sort_values('influence')
print(gg.to_string(columns=['factionName','systemName','influence','vulnerable']))


print()
print("In other words: there's some low-hanging fruit to pick out there.")


print()
print("You said you just wanted something to shoot at.  Go shoot 'em.  I should have the new system")
print("online in a few days.  Let me know if you want another update after we xxxxxxx the new terminal")
print("tonight.  In the meantime, I'm gonna cause some trouble out here in the Pleiades.")

print()
print("Fly dangerous,")
print("-E")

# Systems with Club-Related Minor Factions (table)

The graph below shows systems where Club factions are operating.  The ball of blue systems is largely Emperor's Grace factions that may or may not have much to do with Club operations.

The table below presents the same data in a tabular format.  


In [0]:
#@title 
@interact_manual(maxInfluence=(0, 100.0, 0.1))
def show_factions_with(IncludeHomeSystems=False, IncludeControlSystems=True, IncludeEmperorsGrace=False, maxInfluence=12.0):
    return df.loc[(~df['isHomeSystem']|IncludeHomeSystems) & (~df['control']|IncludeControlSystems) & ( ~df[ 'factionName' ].str.contains("Emperor's Grace")|IncludeEmperorsGrace ) & (df['influence'] <= maxInfluence)]

# More crunching

# Galactic Areas of Club Influence

In [0]:
#@title 


res = [i + " in " + j for i, j in zip(filter['factionName'], filter['systemName'])] 
#%run TwoDimensionalRegionsPlot.py

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=filter['x'],
    y=filter['y'],
    mode="markers",  hovertext= res ))

fig.add_trace(go.Scatter(
    x=[-270, -270,  -95,  140,   -20,   -75,   70,   275,   30],
    y=[-200, -50,   150,  120,     0,  -175,   80,   -80,   120],
    mode="text",
    name="Regions",
    text=["Turner Reach", "Sirius Reach", "Corporate Way","Sirius East","Sirius Core","Pleiades", "The Old Worlds", "Gallant Beach", "Bentonia"],
    textposition="bottom center"
))

# Add shapes
fig.update_layout(
    shapes=[
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-350,
            y0=-175,
            x1= -175,
            y1= -250,
            opacity=0.2,
            fillcolor="blue",
            line_color="blue",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-375,
            y0=-100,
            x1= -125,
            y1= 50,
            opacity=0.2,
            fillcolor="yellow",
            line_color="yellow",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-110,
            y0=75,
            x1= -75,
            y1= 130,
            opacity=0.2,
            fillcolor="green",
            line_color="green",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=120,
            y0=-20,
            x1= 160,
            y1= 100,
            opacity=0.2,
            fillcolor="yellow",
            line_color="yellow",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-40,
            y0= -30,
            x1= 50,
            y1= 30,
            opacity=0.2,
            fillcolor="yellow",
            line_color="yellow",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-100,
            y0= -175,
            x1= -50,
            y1= -75,
            opacity=0.2,
            fillcolor="orange",
            line_color="orange",
        ),
       dict(
            type="rect",
            xref="x",
            yref="y",
            x0=  60,
            y0= 10,
            x1= 80,
            y1= 60,
            opacity=0.2,
            fillcolor="orange",
            line_color="orange",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=  250,
            y0= -150,
            x1= 300,
            y1= -100,
            opacity=0.2,
            fillcolor="orange",
            line_color="orange",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=  25,
            y0= 135,
            x1= 40,
            y1= 175,
            opacity=0.2,
            fillcolor="orange",
            line_color="orange",
        ),
    ],
)
fig.update_layout(showlegend=False)




In [0]:
#@title 
res = [i + " in " + j for i, j in zip(filter['factionName'], filter['systemName'])] 
#%run TwoDimensionalRegionsPlot.py

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=filter['x'],
    y=filter['z'],
    mode="markers",  hovertext= res ))

fig.add_trace(go.Scatter(
    x=[-270, -270,  -95,  140,   -20,   -75,   70,   275,   30],
    y=[-200, -50,   150,  120,     0,  -175,   80,   -80,   120],
    mode="text",
    name="Regions",
    text=["Turner Reach", "Sirius Reach", "Corporate Way","Sirius East","Sirius Core","Pleiades", "The Old Worlds", "Gallant Beach", "Bentonia"],
    textposition="bottom center"
))

# Add shapes
fig.update_layout(
    shapes=[
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-350,
            y0=-175,
            x1= -175,
            y1= -250,
            opacity=0.2,
            fillcolor="blue",
            line_color="blue",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-375,
            y0=-100,
            x1= -125,
            y1= 50,
            opacity=0.2,
            fillcolor="yellow",
            line_color="yellow",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-110,
            y0=75,
            x1= -75,
            y1= 130,
            opacity=0.2,
            fillcolor="green",
            line_color="green",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=120,
            y0=-80,
            x1= 160,
            y1= 130,
            opacity=0.2,
            fillcolor="yellow",
            line_color="yellow",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-40,
            y0= -30,
            x1= 50,
            y1= 30,
            opacity=0.2,
            fillcolor="yellow",
            line_color="yellow",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-100,
            y0= -400,
            x1= -50,
            y1= -200,
            opacity=0.2,
            fillcolor="orange",
            line_color="orange",
        ),
       dict(
            type="rect",
            xref="x",
            yref="y",
            x0=  60,
            y0= 10,
            x1= 80,
            y1= 60,
            opacity=0.2,
            fillcolor="orange",
            line_color="orange",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=  250,
            y0= -40,
            x1= 300,
            y1= -0,
            opacity=0.2,
            fillcolor="orange",
            line_color="orange",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=  25,
            y0= 135,
            x1= 40,
            y1= 175,
            opacity=0.2,
            fillcolor="orange",
            line_color="orange",
        ),
    ],
)
fig.update_layout(showlegend=False)

# FAQ

**What is this?**

1. Welcome to Club Raiders.
2. 'The Club' or 'The Cabal' is a mysterious group in the game Elite: Dangerous.
3. The Club is apparently related to two other long-standing mysteries in Elite: Dangerous - Raxxla and The Dark Wheel.
4. Almost everything we know about The Club is conjecture.
5. A group in Elite Dangerous known as The Children of Raxxla has provided reliable information about other in-game mysteries in the past.
6. The Children of Raxxla have released documents about The Club.  This information may or may not be reliable.
7. Two of these documents are known as The Holdstock Report and The Club Unmasked.
8. These two documents suggest that some in-game minor factions support The Club, and others oppose them.

This document considers the factions identified on the chart below, in particular where these factions are operating throughout the Milky Way.

**What are the connections between these factions and the Club?**

![Big Crazy Club Diagram](https://github.com/HausReport/ClubRaiders/blob/master/info/My%20Club%20Diagram.png?raw=true)

Helping individual players and groups find Club factions operating in their vicinity is part one of the Bolan Plan: **Identify, Engage, Eradicate**.


# Utilities

## All Club Factions as Comma-Delimited Text

If you like to work with data, you might find the comma-delimited text below useful:


In [0]:
#@title 
for xcs in club_systems_arr:
    cs: FactionInstance = xcs
    #if( "Emperor's Grace" in cs.get_name() ): continue
    #vulnerable = cs.isVulnerable()
    #if (vulnerable == False): continue
    cs.printCSV()

## Hic Sunt Dracones

In [0]:
combo = widgets.Combobox(
    # value='John',
    placeholder='Choose Someone',
    options=['Paul', 'John', 'George', 'Geremy'],
    description='Combobox:',
    ensure_option=True,
    disabled=False
)
output = widgets.Output()

display(combo, output)

def on_value_change(change):
        print(change['new'])
combo.observe(on_value_change, names='value')

In [0]:
a = -69.0625
b = 22.37
c = -148.09375
dist = 150
tit = f'Club systems within {dist}ly of ({a},{b},{b})'
#a=-80.15625
#b=144.09375
#c=-333.375
#hover_template={'%{systemName}\n%{factionName'},
fie = df.query(f'sqrt( (x-{a})**2 + (y-{b})**2 + (z-{c})**2)<{dist}')
figz = px.scatter_3d(fie, x='x', y='z', z='y', 
                    color='factionName', 
                     title=tit,
                     hover_name= 'systemName',
                     hover_data= fie.keys())
figz.show()
#hover_data = {'systemName','factionName','influence'},


In [0]:
df.keys()

In [0]:
pfs = ret[1]

In [0]:
combo = widgets.Combobox(
    # value='John',
    placeholder='Choose a Player Faction',
    options= list(pfs.keys()), #nameArray, #[('Galileo', 0), ('Brahe', 1), ('Hubble', 2)], #[('1',1),('2',2),('3',3)], #pfs,
    description='Faction:',
    ensure_option=True,
    disabled=False
)
output = widgets.Output()

display(combo, output)

def on_value_change(change):
        print(pfs[change['new']])
combo.observe(on_value_change, names='value')

In [0]:
sss = ret[2]

In [0]:
combo = widgets.Combobox(
    # value='John',
    placeholder='Choose Inhabited System',
    options= list(sss.keys()), #nameArray, #[('Galileo', 0), ('Brahe', 1), ('Hubble', 2)], #[('1',1),('2',2),('3',3)], #pfs,
    description='System:',
    ensure_option=True,
    disabled=False
)
output = widgets.Output()

display(combo, output)

def on_value_change(change):
        print(sss[change['new']])
combo.observe(on_value_change, names='value')

In [0]:
#
# System chooser
#
combo1 = widgets.Combobox(
    # value='John',
    placeholder='Choose Inhabited System',
    options= list(sss.keys()), #nameArray, #[('Galileo', 0), ('Brahe', 1), ('Hubble', 2)], #[('1',1),('2',2),('3',3)], #pfs,
    description='System:',
    ensure_option=True,
    disabled=False
)

#
# PMF chooser
#
combo2 = widgets.Combobox(
    # value='John',
    placeholder='Choose Player Faction',
    options= list(pfs.keys()), #nameArray, #[('Galileo', 0), ('Brahe', 1), ('Hubble', 2)], #[('1',1),('2',2),('3',3)], #pfs,
    description='Player Faction:',
    ensure_option=True,
    disabled=False
)

output = widgets.Output()
display(combo1, output)
display(combo2, output)

#initialized: bool = False
#container : px.scatter_3d = None

sysName = 'Alioth'
sys = sss[sysName]
a = sys[0]
b = sys[1]
c = sys[2]
dist = 150
tit = f'Club systems within {dist}ly of {sysName}'
#fie = df.query(f'sqrt( (x-{a})**2 + (y-{b})**2 + (z-{c})**2)<{dist}')
figz = px.scatter_3d(filter, x='x', y='z', z='y',color='influence', hover_name= 'systemName', hover_data= fie.keys(),title=tit)
figz.show()

def updateGraph(sysName):
  sys = sss[sysName]
  a = sys[0]
  b = sys[1]
  c = sys[2]
  dist = 150
  tit = f'Club systems within {dist}ly of {sysName}'
  figz.update_xaxes(range=[100, 200])
  figz.update_yaxes(range=[100, 200])
  figz.update_xaxes(range=[100, 200])
  layout=go.Layout(height=800, width=800, title='Examining Population and Life Expectancy Over Time', yaxis=dict(range=[500, 700]), xaxis=dict(range=[100, 200]))


  figz.update_layout(  layout)
  figz.show()
  #yaxis=dict(range=[500, 700]), xaxis=dict(range=[100, 200]), title="Hi!"  )
  print("Hi!")


def on_system_change(change):
  print("Updating graph")
  updateGraph(change['new'])

#updateGraph('Alioth')
combo1.observe(on_system_change, names='value')
combo2.observe(on_system_change, names='value')

In [0]:


@interact_manual(maxInfluence=(0, 100.0, 0.1))
def show_factions_with(IncludeHomeSystems=False, IncludeControlSystems=True, IncludeEmperorsGrace=False, maxInfluence=12.0):
    dat0 = df.loc[(~df['isHomeSystem']|IncludeHomeSystems) & (~df['control']|IncludeControlSystems) & ( ~df[ 'factionName' ].str.contains("Emperor's Grace")|IncludeEmperorsGrace ) & (df['influence'] <= maxInfluence)]
    sysName = 'Alioth'
    sys = sss[sysName]
    a = sys[0]
    b = sys[1]
    c = sys[2]
    dist = 150
    tit = f'Club systems within {dist}ly of {sysName}'
    #figz.data[0].color = 'population'
    camera = dict(center=dict(x=500, y=1000, z=4))
    figlol.update_layout(      scene_camera=camera, title="gahhh")
    figlol.show()
    print("H1")
    #fie = dat0.query(f'sqrt( (x-{a})**2 + (y-{b})**2 + (z-{c})**2)<{dist}')

    


figlol = px.scatter_3d(filter, x='x', y='z', z='y',color='influence', hover_name= 'systemName', hover_data= fie.keys(),title=tit)
figlol.show()
#figz.update(titl )


In [0]:
import plotly.graph_objects as go
import numpy as np

x0=10
y0=10
z0=10

dx=1
dy=1
dz=1

xa=[0, 1, 1, 0, 0, 1, 1, 0]
ya=[0, 1, 1, 0, 0, 1, 1, 0]
za=[0, 0, 0, 0, 1, 1, 1, 1]

xl = [t * dx + x0 for t in xa]
yl = [t * dy + y0 for t in ya]
zl = [t * dz + z0 for t in za]

fign = go.Figure(data=[
    go.Mesh3d(
        # 8 vertices of a cube
        x=xl,
        y=yl,
        z=zl,
        color = 'gold',
        opacity=0.4,
        # i, j and k give the vertices of triangles
        i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
        j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
        k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
        name='y',
        showscale=True
    )
])

fign.show()

In [0]:
l = [0, 1, 1, 0, 0, 1, 1, 0]
l = [x * 5+ 7 for x in l]
print(str(l))

In [0]:
def getMesh( x0, y0, z0, dx, dy, dz):
      x=[0, 1, 1, 0, 0, 1, 1, 0]
      y=[0, 1, 1, 0, 0, 1, 1, 0]
      z=[0, 0, 0, 0, 1, 1, 1, 1]

      x = [t * dx + x0 for t in x]
      y = [t * dy + y0 for t in y]
      z = [t * dz + z0 for t in z]

      return go.Mesh3d(
        # 8 vertices of a cube
        x,
        y,
        z,
        i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
        j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
        k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
        opacity=0.4,
        color='cyan')